# Understanding the dataset

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [21]:
data_file = pd.read_csv('dataset/combined_dataset.csv')

In [22]:
data_file.head(5)

,domain,ranking,isIp,valid,activeDuration,urlLen,is@,isredirect,haveDash,domainLen,nosOfSubdomain,label
0,www.voting-yahoo.com,10000000,0,0,0,20,0,0,1,20,2,1
1,www.zvon.org/xxl/WSDL1.1/Output/index.html,194914,0,1,7305,42,0,0,0,12,2,0
2,tecportais.com/file-security-update-infonfmati...,10000000,0,0,0,155,0,0,0,14,1,1
3,bima.astro.umd.edu/nemo/linuxastro/,7001,0,0,0,35,0,0,0,18,3,0
4,huarui-tec.com/js/?us.battle.net/login/en/?ref...,10000000,0,1,730,79,0,0,1,14,1,1


In [23]:
#Checking for null values
data_file.isna().any()

domain            False
ranking           False
isIp              False
valid             False
activeDuration    False
urlLen            False
is@               False
isredirect        False
haveDash          False
domainLen         False
nosOfSubdomain    False
label             False
dtype: bool

In [24]:
#Checking the distribution of data
print(data_file['label'].value_counts())
print(data_file.shape)

1    55914
0    39996
Name: label, dtype: int64
(95910, 12)


# Cleaning and preprocessing of Questions

### Preprocessing

1. Remove Spcial characters from domain
2. Convert all the characters into small letters

In [44]:
def remove_characters(row):
    row = ''.join(e for e in str(row) if e.isalnum())
    return row

data_file['domain'] = data_file['domain'].apply(remove_characters)

In [45]:
data_file.head(5)

,domain,ranking,isIp,valid,activeDuration,urlLen,is@,isredirect,haveDash,domainLen,nosOfSubdomain,label
0,wwwvotingyahoocom,10000000,0,0,0,20,0,0,1,20,2,1
1,wwwzvonorgxxlWSDL11Outputindexhtml,194914,0,1,7305,42,0,0,0,12,2,0
2,tecportaiscomfilesecurityupdateinfonfmationppl...,10000000,0,0,0,155,0,0,0,14,1,1
3,bimaastroumdedunemolinuxastro,7001,0,0,0,35,0,0,0,18,3,0
4,huaruiteccomjsusbattlenetloginenrefgofcuyeusba...,10000000,0,1,730,79,0,0,1,14,1,1


In [46]:
data_file['domain'].value_counts()

bjcuriocomjsindexhtmappAAAAAAAAAAAAAAAAAA                                                                                                          7
krotzonbizpCntrldewebscrphpcmdsessionstartampsdr01aGVsZ2EucG9ldHNjaEB0LW9ubGluZS5kZQ                                                               6
kksamararufidelidadetamcombrjava                                                                                                                   6
wwwbjcuriocomjsindexhtmusbattlenetloginenrefmvexylnusbattlenetd3enindex                                                                            6
posteitqwertyatworkbeingbangkokmagazinecomvbvindexphpidcXdlcnR5YXR3b3JrQHlhaG9vLml0Cg                                                              6
                                                                                                                                                  ..
wwwvantagensfidelidadecomcadastrophp                                                                      